In [1]:
# batch gradient descent
def update_parameters_with_gd(parameters, grads, learning_rate):
    L = len(parameters)//2
    
    for l in range(L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate*grads['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate*grads['db'+str(l+1)]
        
    return parameters

In [4]:
# batch gradient descent and stohastic gradient descent are different from each other
# batch gradient descent trains whole training set while SGD trains only one training example at a time

# BGD
### X = data_input
### Y = labels
### parameters = initialize_parameters(layers_dims)
### for i in range(0,num_iterations):
###     a, caches = forward_propagation(X, parameters)
###     cost = compute_cost(a, Y)
###     grads = backward_propagation(a, caches, parameters)
###     parameters = update_parameters(parameters, grads)
    
# SGD
### X = data_input
### Y = labels
### parameters = initialize_parameters(layers_dims)
### for i in range(0, num_iterations):
###     for j in range(0, m):
###         a, caches = forward_propagation(X[:,j], parameters)
###         cost = compute_cost(a, Y[:,j])
###         grads = backward_propagation(a, caches, parameters)
###         parameters = update_parameters(parameters, grads)

In [6]:
# bulid mini-batches
# need shuffle and partition
def random_mini_batches(X, Y, mini_batch_size = 64):
    m = X.shape[1]
    mini_batches = []
    
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))
    
    num_complete_minibatches = math.floor(m/mini_batch_size)
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:,k*mini_batch_size:(k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k*mini_batch_size:(k+1)*mini_batch_size]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:,num_complete_minibatches*mini_batch_size:m]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches*mini_batch_size:m]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
        
    return mini_batches
# size of mini_batch id always choose powers of 2 大概是和内存有关系

In [7]:
# momentum
def initialize_volecity(parameters):
    L = len(parameters)//2
    v = {}
    
    for l in range(L):
        v['dW'+str(l+1)] = np.zeros(parameters['W'+str(l+1)].shape)
        v['db'+str(l+1)] = np.zeros(parameters['b'+str(l+1)].shape)
        
    return v

In [9]:
# the larger beta is, the smoother the update
# beta usually between 0.8 to 0.999, default is always to be 0.9
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    L = len(parameters)//2
    
    for l in range(L):
        v['dW'+str(l+1)] = beta*v['dW'+str(l+1)] + (1-beta)*grads['dW'+str(l+1)]
        v['db'+str(l+1)] = beta*v['db'+str(l+1)] + (1-beta)*grads['db'+str(l+1)]
        
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate*v['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate*v['db'+str(l+1)]
        
    return parameters, v

In [10]:
# Adam
def innitialize_adam(parameters):
    L = len(parameters)//2
    v = {}
    s = {}
    
    for l in range(L):
        v['dW'+str(l+1)] = np.zeros(parameters['W'+str(l+1)].shape)
        v['db'+str(l+1)] = np.zeros(parameters['b'+str(l+1)].shape)
        s['dW'+str(l+1)] = np.zeros(parameters['W'+str(l+1)].shape)
        s['db'+str(l+1)] = np.zeros(parameters['b'+str(l+1)].shape)
    return v, s

In [11]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
    L = len(parameters)//2
    v_corrected = {}
    s_corrected = {}
    
    for l in range(L):
        v['dW'+str(l+1)] = beta1*v['dW'+str(l+1)] + (1-beta1)*grads['dW'+str(l+1)]
        v['db'+str(l+1)] = beta1*v['db'+str(l+1)] + (1-beta1)*grads['db'+str(l+1)]
        
        v_corrected['dW'+str(l+1)] = v['dW'+str(l+1)]/(1-beta1**t)
        v_corrected["db" + str(l+1)] = v['db'+str(l+1)]/(1-beta1**t)
        
        s["dW" + str(l+1)] = beta2*s['dW'+str(l+1)] + (1-beta2)*(grads['dW'+str(l+1)]**2)
        s["db" + str(l+1)] = beta2*s['db'+str(l+1)] + (1-beta2)*(grads['db'+str(l+1)]**2)
        
        s_corrected["dW" + str(l+1)] = s['dW'+str(l+1)]/(1-beta2**t)
        s_corrected["db" + str(l+1)] = s['db'+str(l+1)]/(1-beta2**t)
        
        parameters["W" + str(l+1)] = parameters['W'+str(l+1)] - learning_rate*v_corrected['dW'+str(l+1)]/(np.sqrt(s_corrected['dW'+str(l+1)])+epsilon)
        parameters["b" + str(l+1)] = parameters['b'+str(l+1)] - learning_rate*v_corrected['db'+str(l+1)]/(np.sqrt(s_corrected['db'+str(l+1)])+epsilon)
        
    return parameters, v, s

In [12]:
def model(X, Y, layers_dims, optimizer, learning_rate = 0.0007, mini_batch_size = 64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 10000, print_cost = True):
    
    L = len(layers_dims)         
    costs = []                      
    t = 0                           
    seed = 10                        
 
    parameters = initialize_parameters(layers_dims)

    if optimizer == "gd":
        pass
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)

    for i in range(num_epochs):
     
        seed = seed + 1
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)

        for minibatch in minibatches:

            (minibatch_X, minibatch_Y) = minibatch

            a3, caches = forward_propagation(minibatch_X, parameters)

            cost = compute_cost(a3, minibatch_Y)

            grads = backward_propagation(minibatch_X, minibatch_Y, caches)

            if optimizer == "gd":
                parameters = update_parameters_with_gd(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters, v = update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
            elif optimizer == "adam":
                t = t + 1 # Adam counter
                parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
        if print_cost and i % 100 == 0:
            costs.append(cost)

    return parameters